In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from IPython.core.display import display, HTML
import re
import json

In [ ]:
baseSite = 'https://bindingofisaacrebirth.fandom.com'
itemsPage = 'https://bindingofisaacrebirth.fandom.com/wiki/Items'

In [ ]:
page = requests.get(itemsPage)

In [ ]:
soup = bs(page.content, 'html.parser')

In [ ]:
pageLinks = set()

for a in soup.find_all('a', href=True):
    if a['href'].startswith('/wiki/'):
        pageLinks.add((baseSite+a['href']))

In [ ]:
print(len(pageLinks))

994


In [ ]:
infoDict = {}
baseImagePath = '/content/drive/MyDrive/IsaacItemID/Icons/'
count = 0 
# for each item description page
for page in pageLinks:

    # get the page
    p = requests.get(page)
    soup = bs(p.content, 'html.parser')
    count += 1
    # get the name of the item
    itemName = page.split('/')[-1]
    # get the image
    for img in soup.find_all('img'):
        try:
            if itemName in img['data-src'] and '128' in img['data-src']:
                if itemName not in infoDict:
                    print(itemName)
                    effects = str(soup.find(id='Effects').parent.find_next('ul'))
          
                # get the effects of the item
                    for link in re.findall('("/wiki/.{1,20}" )',effects):
                        newLink = '"'+baseSite+link[1:]
                        effects = effects.replace(link,newLink)

                    # set the file name of the image
                    fileName = itemName+'.png'

                    # set up the full path to the image
                    fullPath = baseImagePath+fileName

                    # get the image
                    image_url = img['data-src']
                    img_data = requests.get(image_url).content
          
                    # save the image
                    with open(fullPath, 'wb') as handler:
                        handler.write(img_data)

                    infoDict[itemName] = {'Effects':effects,'ImagePath':fullPath}
                    if count%25 == 0:
                        print('---------------------------------------------------')
                        print(count,'/',len(pageLinks),(count/len(pageLinks)*100),'% complete')
                        print('---------------------------------------------------')
        
        except:
            pass

In [ ]:
infoDict['Hushy']['ImagePath']

'/content/drive/MyDrive/IsaacItemID/Icons/Hushy.png'

In [ ]:
# save json

with open('/content/drive/MyDrive/IsaacItemID/ItemDictionary.json', 'w') as fp:
    json.dump(infoDict, fp)